In [2]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1500)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [3]:

print(tf.__version__)
print(tf.keras.__version__)

2.0.0-beta1
2.2.4-tf


In [4]:
#最常见的模型类型是层的堆叠：tf.keras.Sequential 模型
model = tf.keras.Sequential()
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))


In [3]:
'''
2.2网络配置
tf.keras.layers中网络配置：

activation：设置层的激活函数。此参数由内置函数的名称指定，或指定为可调用对象。默认情况下，系统不会应用任何激活函数。

kernel_initializer 和 bias_initializer：创建层权重（核和偏差）的初始化方案。此参数是一个名称或可调用对象，默认为 “Glorot uniform” 初始化器。

kernel_regularizer 和 bias_regularizer：应用层权重（核和偏差）的正则化方案，例如 L1 或 L2 正则化。默认情况下，系统不会应用正则化函数。
'''
# Create a sigmoid layer:
layers.Dense(64, activation='sigmoid')
# Or:
layers.Dense(64, activation=tf.sigmoid)

# A linear layer with L1 regularization of factor 0.01 applied to the kernel matrix:
layers.Dense(64, kernel_regularizer=tf.keras.regularizers.l1(0.01))

# A linear layer with L2 regularization of factor 0.01 applied to the bias vector:
layers.Dense(64, bias_regularizer=tf.keras.regularizers.l2(0.01))

# A linear layer with a kernel initialized to a random orthogonal matrix:
layers.Dense(64, kernel_initializer='orthogonal')

# A linear layer with a bias vector initialized to 2.0s:
layers.Dense(64, bias_initializer=tf.keras.initializers.constant(2.0))

In [5]:
'''
3.训练和评估
3.1设置训练流程
构建好模型后，通过调用 compile 方法配置该模型的学习流程：
'''
##可以model.add()添加层，也可以直接将列表加进去

model = tf.keras.Sequential([
# Adds a densely-connected layer with 64 units to the model:
layers.Dense(64, activation='relu'),
# Add another:
layers.Dense(64, activation='relu'),
# Add a softmax layer with 10 output units:
layers.Dense(10, activation='softmax')])


model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              ##上面的优化函数才是对的，tf.train.AdamOptimizer不能用
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [5]:
#  optimizer=tf.optimizers.Adam(0.001)
#  optimizer2=tf.keras.optimizers.Adam(0.001)

#两个来源一样都是keras.adam


In [6]:
# Configure a model for mean-squared error regression.
model.compile(optimizer=tf.keras.optimizers.Adam(0.01),
              loss='mse',       # mean squared error
              metrics=['mae'])  # mean absolute error

# Configure a model for categorical classification.
model.compile(optimizer=tf.keras.optimizers.RMSprop(0.01),#tf.train.RMSPropOptimizer(0.01)不能用
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=[tf.keras.metrics.categorical_accuracy])

In [6]:
#3.2输入Numpy数据
import numpy as np

train_x = np.random.random((1000, 72))
train_y = np.random.random((1000, 10))

val_x = np.random.random((200, 72))
val_y = np.random.random((200, 10))

model.fit(train_x, train_y, epochs=10, batch_size=100,
          validation_data=(val_x, val_y))


W0823 08:06:48.920107 140578539656960 deprecation.py:323] From /usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 1000 samples, validate on 200 samples
Epoch 1/10
1000/1000 [==============================] - 1s 554us/sample - loss: 11.6933 - accuracy: 0.1030 - val_loss: 11.7745 - val_accuracy: 0.1200
Epoch 2/10
1000/1000 [==============================] - 0s 67us/sample - loss: 11.6446 - accuracy: 0.1180 - val_loss: 11.7627 - val_accuracy: 0.0650
Epoch 3/10
1000/1000 [==============================] - 0s 58us/sample - loss: 11.6303 - accuracy: 0.1190 - val_loss: 11.7594 - val_accuracy: 0.0600
Epoch 4/10
1000/1000 [==============================] - 0s 54us/sample - loss: 11.6226 - accuracy: 0.1070 - val_loss: 11.7585 - val_accuracy: 0.0750
Epoch 5/10
1000/1000 [==============================] - 0s 61us/sample - loss: 11.6170 - accuracy: 0.1270 - val_loss: 11.7570 - val_accuracy: 0.0950
Epoch 6/10
1000/1000 [==============================] - 0s 53us/sample - loss: 11.6127 - accuracy: 0.1270 - val_loss: 11.7562 - val_accuracy: 0.0600
Epoch 7/10
1000/1000 [==============================] - 0s

In [7]:
#3.3tf.data输入数据
dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y))
dataset = dataset.batch(32)
dataset = dataset.repeat()
val_dataset = tf.data.Dataset.from_tensor_slices((val_x, val_y))
val_dataset = val_dataset.batch(32)
val_dataset = val_dataset.repeat()

model.fit(dataset, epochs=10, steps_per_epoch=30,
          validation_data=val_dataset, validation_steps=3)


W0823 08:07:39.601559 140578539656960 training_utils.py:1436] Expected a shuffled dataset but input dataset `x` is not shuffled. Please invoke `shuffle()` on input dataset.


Epoch 1/10
30/30 [==============================] - 0s 12ms/step - loss: 11.6008 - accuracy: 0.1469 - val_loss: 11.9940 - val_accuracy: 0.0833
Epoch 2/10
30/30 [==============================] - 0s 11ms/step - loss: 11.5828 - accuracy: 0.1389 - val_loss: 11.9843 - val_accuracy: 0.0938
Epoch 3/10
30/30 [==============================] - 0s 7ms/step - loss: 11.5938 - accuracy: 0.1538 - val_loss: 11.9902 - val_accuracy: 0.0833
Epoch 4/10
30/30 [==============================] - 0s 7ms/step - loss: 11.6045 - accuracy: 0.1688 - val_loss: 11.9958 - val_accuracy: 0.0729
Epoch 5/10
30/30 [==============================] - 0s 7ms/step - loss: 11.5691 - accuracy: 0.1859 - val_loss: 12.0027 - val_accuracy: 0.0938
Epoch 6/10
30/30 [==============================] - 0s 7ms/step - loss: 11.5686 - accuracy: 0.1891 - val_loss: 12.0048 - val_accuracy: 0.1146
Epoch 7/10
30/30 [==============================] - 0s 7ms/step - loss: 11.5349 - accuracy: 0.1976 - val_loss: 12.0074 - val_accuracy: 0.1250
Epoc

In [14]:
#3.4评估与预测
test_x = np.random.random((1000, 72))
test_y = np.random.random((1000, 10))
model.evaluate(test_x, test_y, batch_size=32)
test_data = tf.data.Dataset.from_tensor_slices((test_x, test_y))
test_data = test_data.batch(32).repeat()
model.evaluate(test_data, steps=30)


30/30 [==============================] - 0s 3ms/step - loss: 11.5784 - categorical_accuracy: 0.1146


[11.57843910853068, 0.114583336]

In [15]:
'''输入 tf.data 数据集
使用 Datasets API 可扩展为大型数据集或多设备训练。
将 tf.data.Dataset 实例传递到 fit 方法：'''
# Instantiates a toy dataset instance:
dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y))
dataset = dataset.batch(32)
dataset = dataset.repeat()

# Don't forget to specify `steps_per_epoch` when calling `fit` on a dataset.
model.fit(dataset, epochs=10, steps_per_epoch=30)

W0708 08:55:04.768955 140074130970368 training_utils.py:1436] Expected a shuffled dataset but input dataset `x` is not shuffled. Please invoke `shuffle()` on input dataset.


Epoch 1/10
30/30 [==============================] - 0s 7ms/step - loss: 11.5401 - categorical_accuracy: 0.1208
Epoch 2/10
30/30 [==============================] - 0s 8ms/step - loss: 11.5336 - categorical_accuracy: 0.0972
Epoch 3/10
30/30 [==============================] - 0s 8ms/step - loss: 11.5608 - categorical_accuracy: 0.1090
Epoch 4/10
30/30 [==============================] - 0s 8ms/step - loss: 11.5334 - categorical_accuracy: 0.1132
Epoch 5/10
30/30 [==============================] - 0s 8ms/step - loss: 11.5519 - categorical_accuracy: 0.1068
Epoch 6/10
30/30 [==============================] - 0s 8ms/step - loss: 11.5244 - categorical_accuracy: 0.1239
Epoch 7/10
30/30 [==============================] - 0s 8ms/step - loss: 11.5345 - categorical_accuracy: 0.1036
Epoch 8/10
30/30 [==============================] - 0s 8ms/step - loss: 11.5216 - categorical_accuracy: 0.1314
Epoch 9/10
30/30 [==============================] - 0s 8ms/step - loss: 11.5612 - categorical_accuracy: 0.1197
E

In [16]:
# predict
result = model.predict(test_x, batch_size=32)
print(result)


[[0.11008526 0.09580339 0.09877068 ... 0.10400292 0.10196316 0.09956846]
 [0.10401383 0.09606762 0.1050818  ... 0.10323157 0.09778462 0.09813404]
 [0.10338207 0.09573887 0.10557301 ... 0.10309436 0.09741373 0.09807102]
 ...
 [0.10336718 0.09577353 0.10557701 ... 0.10310866 0.09741092 0.09813978]
 [0.10907789 0.0865532  0.09761617 ... 0.0987197  0.11294651 0.10839939]
 [0.10457279 0.09494692 0.1042695  ... 0.10273406 0.09940954 0.09927887]]


In [17]:
'''
4.构建高级模型
4.1函数式api
tf.keras.Sequential 模型是层的简单堆叠，无法表示任意模型。使用 Keras 函数式 API 可以构建复杂的模型拓扑，例如：

多输入模型，

多输出模型，

具有共享层的模型（同一层被调用多次），

具有非序列数据流的模型（例如，残差连接）。


使用函数式 API 构建的模型具有以下特征：

层实例可调用并返回张量。
输入张量和输出张量用于定义 tf.keras.Model 实例。
此模型的训练方式和 Sequential 模型一样。

##tf.keras.Input中shape表示去掉batch_size之后的维度；
train_x=（1000,72）
'''
input_x = tf.keras.Input(shape=(72,))
hidden1 = layers.Dense(32, activation='relu')(input_x)
hidden2 = layers.Dense(16, activation='relu')(hidden1)
pred = layers.Dense(10, activation='softmax')(hidden2)

model = tf.keras.Model(inputs=input_x, outputs=pred)
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
             loss=tf.keras.losses.categorical_crossentropy,
             metrics=['accuracy'])
model.fit(train_x, train_y, batch_size=32, epochs=5)


Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 430us/sample - loss: 11.6284 - accuracy: 0.0990
Epoch 2/5
1000/1000 [==============================] - 0s 174us/sample - loss: 11.5598 - accuracy: 0.1050
Epoch 3/5
1000/1000 [==============================] - 0s 180us/sample - loss: 11.5571 - accuracy: 0.1040
Epoch 4/5
1000/1000 [==============================] - 0s 187us/sample - loss: 11.5562 - accuracy: 0.1020
Epoch 5/5
1000/1000 [==============================] - 0s 164us/sample - loss: 11.5557 - accuracy: 0.1040


In [19]:
print(train_x.shape)
print(train_y.shape)
      

(1000, 72)
(1000, 10)


In [20]:
'''
4.2模型子类化
通过对 tf.keras.Model 进行子类化并定义您自己的前向传播来构建完全可自定义的模型。
在 init 方法中创建层并将它们设置为类实例的属性。在 call 方法中定义前向传播
'''

class MyModel(tf.keras.Model):
    #在 init 方法中创建层并将它们设置为类实例的属性
    def __init__(self, num_classes=10):
        super(MyModel, self).__init__(name='my_model')
        self.num_classes = num_classes
        self.layer1 = layers.Dense(32, activation='relu')
        self.layer2 = layers.Dense(num_classes, activation='softmax')
    #在 call 方法中定义前向传播
    def call(self, inputs):
        h1 = self.layer1(inputs)
        out = self.layer2(h1)
        return out
    
    def compute_output_shape(self, input_shape):
        shape = tf.TensorShapej(input_shape).as_list()
        shape[-1] = self.num_classes
        return tf.TensorShape(shape)

model = MyModel(num_classes=10)
model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
             loss=tf.keras.losses.categorical_crossentropy,
             metrics=['accuracy'])

model.fit(train_x, train_y, batch_size=16, epochs=5)


Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 1s 540us/sample - loss: 11.7061 - accuracy: 0.1190
Epoch 2/5
1000/1000 [==============================] - 0s 334us/sample - loss: 11.6337 - accuracy: 0.1220
Epoch 3/5
1000/1000 [==============================] - 0s 329us/sample - loss: 11.6060 - accuracy: 0.1170
Epoch 4/5
1000/1000 [==============================] - 0s 315us/sample - loss: 11.5845 - accuracy: 0.1170
Epoch 5/5
1000/1000 [==============================] - 0s 284us/sample - loss: 11.5724 - accuracy: 0.1050


In [24]:
'''
4.3自定义层
通过对 tf.keras.layers.Layer 进行子类化并实现以下方法来创建自定义层：
使用 add_weight 方法添加权重。


'''
class MyLayer(layers.Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(MyLayer, self).__init__(**kwargs)
    

    #build：创建层的权重。##不知道输入的维度重新build函数
    def build(self, input_shape):
        shape = tf.TensorShape((input_shape[1], self.output_dim))
        self.kernel = self.add_weight(name='kernel1', shape=shape,
                                   initializer='uniform', trainable=True)
        super(MyLayer, self).build(input_shape)
    #call：定义前向传播。将输入和该层的权重联系起来（相乘）
    def call(self, inputs):
        return tf.matmul(inputs, self.kernel)

    #compute_output_shape：指定在给定输入形状的情况下如何计算层的输出形状。
    #或者，可以通过实现 get_config 方法和 from_config 类方法序列化层。
    def compute_output_shape(self, input_shape):
        shape = tf.TensorShape(input_shape).as_list()
        shape[-1] = self.output_dim
        return tf.TensorShape(shape)

    def get_config(self):
        base_config = super(MyLayer, self).get_config()
        base_config['output_dim'] = self.output_dim
        return base_config

    @classmethod
    def from_config(cls, config):
        return cls(**config)
    
model = tf.keras.Sequential(
[
    MyLayer(10),
    layers.Activation('softmax')
])


model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
             loss=tf.keras.losses.categorical_crossentropy,
             metrics=['accuracy'])

model.fit(train_x, train_y, batch_size=16, epochs=5)

        



Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 397us/sample - loss: 11.5760 - accuracy: 0.0750
Epoch 2/5
1000/1000 [==============================] - 0s 271us/sample - loss: 11.5662 - accuracy: 0.0890
Epoch 3/5
1000/1000 [==============================] - 0s 264us/sample - loss: 11.5582 - accuracy: 0.1240
Epoch 4/5
1000/1000 [==============================] - 0s 260us/sample - loss: 11.5534 - accuracy: 0.1060
Epoch 5/5
1000/1000 [==============================] - 0s 259us/sample - loss: 11.5493 - accuracy: 0.1240


In [ ]:
#当定义网络时不知道网络的维度是可以重写build()函数，用获得的shape构建网络
class MyLayer(layers.Layer):##初始化时不知道输入的维度，此时要重写build函数，自己添加输入的维度
    def __init__(self, unit=32):
        super(MyLayer, self).__init__()
        self.unit = unit
        
    def build(self, input_shape):
        self.weight = self.add_weight(shape=(input_shape[-1], self.unit),
                                     initializer=keras.initializers.RandomNormal(),
                                     trainable=True)
        self.bias = self.add_weight(shape=(self.unit,),
                                   initializer=keras.initializers.Zeros(),
                                   trainable=True)
    
    def call(self, inputs):
        return tf.matmul(inputs, self.weight) + self.bias
        

my_layer = MyLayer(3)
x = tf.ones((3,5))
out = my_layer(x)
print(out)
my_layer = MyLayer(3)

x = tf.ones((2,2))
out = my_layer(x)
print(out)


In [ ]:
# 定义网络层就是：设置网络权重和输出到输入的计算过程
class MyLayer(layers.Layer):
    def __init__(self, input_dim=32, unit=32):
        super(MyLayer, self).__init__()
        
        w_init = tf.random_normal_initializer()
        self.weight = tf.Variable(initial_value=w_init(
            shape=(input_dim, unit), dtype=tf.float32), trainable=True)
        
        b_init = tf.zeros_initializer()
        self.bias = tf.Variable(initial_value=b_init(
            shape=(unit,), dtype=tf.float32), trainable=True)
    
    def call(self, inputs):
        return tf.matmul(inputs, self.weight) + self.bias
        
x = tf.ones((3,5))
my_layer = MyLayer(5, 4)
out = my_layer(x)
print(out)
        

In [ ]:
#按上面构建网络层，图层会自动跟踪权重w和b，当然我们也可以直接用add_weight的方法构建权重
class MyLayer(layers.Layer):
    def __init__(self, input_dim=32, unit=32):
        super(MyLayer, self).__init__()
        self.weight = self.add_weight(shape=(input_dim, unit),
                                     initializer=keras.initializers.RandomNormal(),
                                     trainable=True)
        self.bias = self.add_weight(shape=(unit,),
                                   initializer=keras.initializers.Zeros(),
                                   trainable=True)
    
    def call(self, inputs):
        return tf.matmul(inputs, self.weight) + self.bias
        
x = tf.ones((3,5))
my_layer = MyLayer(5, 4)
out = my_layer(x)
print(out)
        

In [ ]:
#也可以设置不可训练的权重
class AddLayer(layers.Layer):
    def __init__(self, input_dim=32):
        super(AddLayer, self).__init__()
        self.sum = self.add_weight(shape=(input_dim,),
                                     initializer=keras.initializers.Zeros(),
                                     trainable=False)##权重不可训练
       
    
    def call(self, inputs):
        self.sum.assign_add(tf.reduce_sum(inputs, axis=0))
        return self.sum
        
x = tf.ones((3,3))
my_layer = AddLayer(3)
out = my_layer(x)
print(out.numpy())
out = my_layer(x)
print(out.numpy())
print('weight:', my_layer.weights)
print('non-trainable weight:', my_layer.non_trainable_weights)
print('trainable weight:', my_layer.trainable_weights)


In [22]:
#4.3回调

##在fit参数上添加
##回调有验证集的early stopping,以及日志的保存
# monitor: 需要监视的量，val_loss，val_acc
# patience: 当验证集上的性能不再提高时，终止训练。当early stop被激活(如发现loss相比上一个epoch训练没有下降)，则经过patience个epoch后停止训练
# verbose: 信息展示模式
# mode: 'auto','min','max'之一，在min模式训练，如果检测值停止下降则终止训练。在max模式下，当检测值不再上升的时候则停止训练。

callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss'),
    tf.keras.callbacks.TensorBoard(log_dir='./logs')
]
model.fit(train_x, train_y, batch_size=16, epochs=5,
         callbacks=callbacks, validation_data=(val_x, val_y))


Train on 1000 samples, validate on 200 samples
Epoch 1/5


W0708 09:12:06.597361 140074130970368 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.198192). Check your callbacks.


  16/1000 [..............................] - ETA: 12s - loss: 11.7791 - accuracy: 0.0625

W0708 09:12:06.621986 140074130970368 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.106626). Check your callbacks.


1000/1000 [==============================] - 1s 616us/sample - loss: 11.5448 - accuracy: 0.1310 - val_loss: 11.5356 - val_accuracy: 0.0800
Epoch 2/5
1000/1000 [==============================] - 0s 310us/sample - loss: 11.5424 - accuracy: 0.1420 - val_loss: 11.5215 - val_accuracy: 0.0650
Epoch 3/5
1000/1000 [==============================] - 0s 275us/sample - loss: 11.5392 - accuracy: 0.1380 - val_loss: 11.5229 - val_accuracy: 0.0700
Epoch 4/5
1000/1000 [==============================] - 0s 285us/sample - loss: 11.5361 - accuracy: 0.1280 - val_loss: 11.5319 - val_accuracy: 0.0700


In [27]:
'''
5保持和恢复
5.1权重保存
'''
model = tf.keras.Sequential([
layers.Dense(64, activation='relu'),
layers.Dense(10, activation='softmax')])

model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
#训练完模型后才有权重可以保存
model.fit(train_x, train_y, batch_size=16, epochs=5)
model.save_weights('./weights/model')
model.load_weights('./weights/model')
model.save_weights('./model.h5')
model.load_weights('./model.h5')



Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 1s 554us/sample - loss: 11.6760 - accuracy: 0.0970
Epoch 2/5
1000/1000 [==============================] - 0s 348us/sample - loss: 11.6054 - accuracy: 0.1070
Epoch 3/5
1000/1000 [==============================] - 0s 338us/sample - loss: 11.5820 - accuracy: 0.1160
Epoch 4/5
1000/1000 [==============================] - 0s 311us/sample - loss: 11.5659 - accuracy: 0.1120
Epoch 5/5
1000/1000 [==============================] - 0s 368us/sample - loss: 11.5547 - accuracy: 0.1210


In [29]:
ls weights/

checkpoint  model.data-00000-of-00002  model.data-00001-of-00002  model.index


In [30]:
#5.2保存网络结构
# 序列化成json
import json
import pprint
json_str = model.to_json()
pprint.pprint(json.loads(json_str))
fresh_model = tf.keras.models.model_from_json(json_str)



{'backend': 'tensorflow',
 'class_name': 'Sequential',
 'config': {'build_input_shape': [None, 72],
            'layers': [{'class_name': 'Dense',
                        'config': {'activation': 'relu',
                                   'activity_regularizer': None,
                                   'bias_constraint': None,
                                   'bias_initializer': {'class_name': 'Zeros',
                                                        'config': {}},
                                   'bias_regularizer': None,
                                   'dtype': 'float32',
                                   'kernel_constraint': None,
                                   'kernel_initializer': {'class_name': 'GlorotUniform',
                                                          'config': {'seed': None}},
                                   'kernel_regularizer': None,
                                   'name': 'dense_26',
                                   'trainable': Tru

In [31]:
# 保持为yaml格式  #需要提前安装pyyaml

yaml_str = model.to_yaml()
print(yaml_str)
fresh_model = tf.keras.models.model_from_yaml(yaml_str)


backend: tensorflow
class_name: Sequential
config:
  build_input_shape: !!python/tuple
  - null
  - 72
  layers:
  - class_name: Dense
    config:
      activation: relu
      activity_regularizer: null
      bias_constraint: null
      bias_initializer:
        class_name: Zeros
        config: {}
      bias_regularizer: null
      dtype: float32
      kernel_constraint: null
      kernel_initializer:
        class_name: GlorotUniform
        config:
          seed: null
      kernel_regularizer: null
      name: dense_26
      trainable: true
      units: 64
      use_bias: true
  - class_name: Dense
    config:
      activation: softmax
      activity_regularizer: null
      bias_constraint: null
      bias_initializer:
        class_name: Zeros
        config: {}
      bias_regularizer: null
      dtype: float32
      kernel_constraint: null
      kernel_initializer:
        class_name: GlorotUniform
        config:
          seed: null
      kernel_regularizer: null
      name: de

/usr/local/lib/python3.5/dist-packages/tensorflow/python/keras/saving/model_config.py:76: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config = yaml.load(yaml_string)


In [32]:
#5.3保存整个模型
model = tf.keras.Sequential([
  layers.Dense(10, activation='softmax', input_shape=(72,)),
  layers.Dense(10, activation='softmax')
])
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_x, train_y, batch_size=32, epochs=5)
model.save('all_model.h5')
model = tf.keras.models.load_model('all_model.h5')



Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 426us/sample - loss: 11.5665 - accuracy: 0.0870
Epoch 2/5
1000/1000 [==============================] - 0s 193us/sample - loss: 11.5587 - accuracy: 0.0980
Epoch 3/5
1000/1000 [==============================] - 0s 202us/sample - loss: 11.5573 - accuracy: 0.1070
Epoch 4/5
1000/1000 [==============================] - 0s 206us/sample - loss: 11.5567 - accuracy: 0.0940
Epoch 5/5
1000/1000 [==============================] - 0s 193us/sample - loss: 11.5560 - accuracy: 0.1090


In [33]:
ls

all_model.h5               logs/                         weights/
keras_example.ipynb        model.h5
keras_example_beta1.ipynb  tensorflow_2_0_example.ipynb


In [34]:
# 6.将keras用于Estimator
# Estimator API 用于针对分布式环境训练模型。
# 它适用于一些行业使用场景，例如用大型数据集进行分布式训练并导出模型以用于生产
model = tf.keras.Sequential([layers.Dense(10,activation='softmax'),
                          layers.Dense(10,activation='softmax')])

model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

estimator = tf.keras.estimator.model_to_estimator(model)



W0708 09:24:12.507230 140074130970368 estimator.py:1811] Using temporary folder as model directory: /tmp/tmppop698bv
